# Week 19: NLP & Alternative Data Theory
## Quantitative Finance Applications

---

### 📚 Learning Objectives

By the end of this module, you will understand:

1. **NLP Fundamentals** - Text preprocessing, tokenization, and representation
2. **Sentiment Analysis** - Dictionary-based and ML approaches for financial text
3. **Alternative Data** - Sources, quality assessment, and alpha generation
4. **Practical Applications** - News-based signals and sentiment factor construction

---

### 🎯 Key Topics

| Topic | Description | Finance Application |
|-------|-------------|---------------------|
| Text Preprocessing | Cleaning, tokenization, normalization | Earnings call analysis |
| BOW/TF-IDF | Document representation | News classification |
| Word Embeddings | Semantic representations | Similar company detection |
| Sentiment Analysis | Polarity scoring | Trading signals |
| Alternative Data | Non-traditional data sources | Alpha generation |

---

### 📊 Why NLP in Quantitative Finance?

> *"The market is driven by two emotions: fear and greed. NLP helps us quantify these emotions."*

**Information Sources:**
- 10-K/10-Q filings (SEC EDGAR)
- Earnings call transcripts
- News articles and press releases
- Social media (Twitter/X, Reddit, StockTwits)
- Analyst reports

---

# Part 1: NLP Fundamentals

## 1.1 Text Preprocessing Pipeline

Text preprocessing is the foundation of any NLP pipeline. Raw text must be transformed into a clean, standardized format before analysis.

### Preprocessing Steps:

```
Raw Text → Lowercase → Remove Punctuation → Tokenize → Remove Stopwords → Stem/Lemmatize → Clean Text
```

### Key Concepts:

| Step | Description | Example |
|------|-------------|---------|
| **Tokenization** | Split text into words/sentences | "Apple rises" → ["Apple", "rises"] |
| **Lowercasing** | Normalize case | "AAPL" → "aapl" |
| **Stopword Removal** | Remove common words | Remove "the", "is", "and" |
| **Stemming** | Reduce to root form (crude) | "running" → "run" |
| **Lemmatization** | Reduce to dictionary form | "better" → "good" |

### Financial Text Challenges:
- **Ticker symbols**: $AAPL, TSLA
- **Numbers and percentages**: 10%, $1.5B
- **Financial jargon**: "bullish", "dovish", "hawkish"
- **Negation handling**: "not good" vs "good"

In [1]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# NLP Libraries
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt_tab', quiet=True)

# Scikit-learn for ML
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

print("✅ All libraries imported successfully!")
print(f"NLTK version: {nltk.__version__}")

ModuleNotFoundError: No module named 'nltk'

In [ ]:
# Text Preprocessing Pipeline Implementation

class FinancialTextPreprocessor:
    """
    Comprehensive text preprocessing pipeline for financial text.
    """
    
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Financial-specific stopwords to keep
        self.financial_keep = {'up', 'down', 'high', 'low', 'above', 'below', 'not'}
        self.stop_words -= self.financial_keep
        
    def clean_text(self, text):
        """Basic text cleaning."""
        # Lowercase
        text = text.lower()
        # Remove URLs
        text = re.sub(r'http\S+|www\S+', '', text)
        # Keep ticker symbols (convert $AAPL to AAPL)
        text = re.sub(r'\$([A-Za-z]+)', r'\1', text)
        # Remove special characters but keep numbers
        text = re.sub(r'[^a-zA-Z0-9\s%.]', '', text)
        # Remove extra whitespace
        text = ' '.join(text.split())
        return text
    
    def tokenize(self, text):
        """Tokenize text into words."""
        return word_tokenize(text)
    
    def remove_stopwords(self, tokens):
        """Remove stopwords while keeping financial terms."""
        return [t for t in tokens if t not in self.stop_words]
    
    def stem(self, tokens):
        """Apply Porter stemming."""
        return [self.stemmer.stem(t) for t in tokens]
    
    def lemmatize(self, tokens):
        """Apply lemmatization."""
        return [self.lemmatizer.lemmatize(t) for t in tokens]
    
    def preprocess(self, text, use_stemming=False):
        """Full preprocessing pipeline."""
        text = self.clean_text(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        
        if use_stemming:
            tokens = self.stem(tokens)
        else:
            tokens = self.lemmatize(tokens)
            
        return ' '.join(tokens)

# Example usage
preprocessor = FinancialTextPreprocessor()

# Sample financial headlines
sample_texts = [
    "Apple $AAPL surges 5% after beating earnings estimates!",
    "Fed signals interest rate cuts are coming in 2024",
    "Tesla's stock is NOT performing well amid EV competition",
    "Goldman Sachs downgrades tech sector to 'Underweight'",
    "Bitcoin crashes below $30,000 on regulatory concerns"
]

print("=" * 60)
print("TEXT PREPROCESSING DEMONSTRATION")
print("=" * 60)

for text in sample_texts:
    cleaned = preprocessor.preprocess(text)
    print(f"\n📝 Original: {text}")
    print(f"✅ Cleaned:  {cleaned}")

## 1.2 Tokenization Deep Dive

### Types of Tokenization:

| Type | Description | Use Case |
|------|-------------|----------|
| **Word Tokenization** | Split by words | General NLP tasks |
| **Sentence Tokenization** | Split by sentences | Document summarization |
| **Subword Tokenization** | Split into subwords | Modern transformers (BPE, WordPiece) |
| **Character Tokenization** | Split by characters | Spelling correction |

### Stemming vs Lemmatization:

**Stemming** (Rule-based, faster):
- "running" → "run"
- "studies" → "studi" ⚠️ (not a real word)
- "better" → "better" (doesn't change)

**Lemmatization** (Dictionary-based, accurate):
- "running" → "run"
- "studies" → "study" ✅
- "better" → "good" ✅

> **💡 Interview Tip**: For financial text, lemmatization is preferred because accuracy matters more than speed. Stemming can produce non-words that lose meaning.

In [ ]:
# Stemming vs Lemmatization Comparison

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

comparison_words = [
    "running", "studies", "better", "earnings", "increasing",
    "diversified", "volatility", "analysts", "underperforming"
]

print("=" * 60)
print("STEMMING VS LEMMATIZATION COMPARISON")
print("=" * 60)
print(f"{'Word':<20} {'Stemmed':<20} {'Lemmatized':<20}")
print("-" * 60)

for word in comparison_words:
    stemmed = stemmer.stem(word)
    lemmatized = lemmatizer.lemmatize(word)
    print(f"{word:<20} {stemmed:<20} {lemmatized:<20}")

# N-grams demonstration
print("\n" + "=" * 60)
print("N-GRAM TOKENIZATION")
print("=" * 60)

sample_sentence = "The Federal Reserve raised interest rates by 25 basis points"

# Unigrams, Bigrams, Trigrams
from nltk import ngrams

tokens = word_tokenize(sample_sentence.lower())

print(f"\n📝 Sentence: {sample_sentence}")
print(f"\n1️⃣ Unigrams: {tokens}")
print(f"\n2️⃣ Bigrams: {list(ngrams(tokens, 2))}")
print(f"\n3️⃣ Trigrams: {list(ngrams(tokens, 3))}")

---

# Part 2: Text Representation

## 2.1 Bag of Words (BoW) Model

The Bag of Words model represents text as a vector of word counts, ignoring word order.

### Mathematical Formulation:

For a vocabulary $V = \{w_1, w_2, ..., w_n\}$ and a document $d$:

$$\text{BoW}(d) = [c(w_1, d), c(w_2, d), ..., c(w_n, d)]$$

where $c(w_i, d)$ is the count of word $w_i$ in document $d$.

### Properties:
- ✅ Simple and interpretable
- ✅ Works well for document classification
- ❌ Ignores word order and context
- ❌ High dimensionality (vocabulary size)
- ❌ Sparse representations

### Example:
```
Doc 1: "Stock price rises"     → [1, 1, 1, 0, 0]
Doc 2: "Stock price falls"     → [1, 1, 0, 1, 0]
Doc 3: "Price rises sharply"   → [0, 1, 1, 0, 1]

Vocabulary: [stock, price, rises, falls, sharply]
```

In [ ]:
# Bag of Words Implementation

# Sample financial news headlines
financial_headlines = [
    "Apple stock surges on strong iPhone sales",
    "Tesla stock falls amid production concerns",
    "Fed raises interest rates by 25 basis points",
    "Goldman Sachs reports record quarterly earnings",
    "Oil prices surge on supply disruption fears",
    "Tech stocks fall on inflation concerns",
    "Apple reports strong quarterly earnings growth",
    "Bitcoin crashes below support level"
]

# Create Bag of Words using CountVectorizer
bow_vectorizer = CountVectorizer(
    stop_words='english',
    max_features=20,  # Limit vocabulary size
    ngram_range=(1, 1)  # Unigrams only
)

bow_matrix = bow_vectorizer.fit_transform(financial_headlines)

# Display results
feature_names = bow_vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=feature_names,
    index=[f"Doc_{i+1}" for i in range(len(financial_headlines))]
)

print("=" * 60)
print("BAG OF WORDS REPRESENTATION")
print("=" * 60)
print(f"\n📊 Vocabulary Size: {len(feature_names)}")
print(f"📊 Document Count: {len(financial_headlines)}")
print(f"📊 Matrix Shape: {bow_matrix.shape}")
print(f"📊 Sparsity: {1 - bow_matrix.nnz / (bow_matrix.shape[0] * bow_matrix.shape[1]):.2%}")

print("\n🔤 Vocabulary:", list(feature_names))
print("\n📋 Document-Term Matrix:")
display(bow_df)

## 2.2 TF-IDF (Term Frequency - Inverse Document Frequency)

TF-IDF addresses a key limitation of BoW: common words get too much weight.

### Mathematical Formulation:

**Term Frequency (TF):**
$$\text{TF}(t, d) = \frac{\text{count of term } t \text{ in document } d}{\text{total terms in document } d}$$

**Inverse Document Frequency (IDF):**
$$\text{IDF}(t, D) = \log\left(\frac{N}{|\{d \in D : t \in d\}|}\right)$$

where $N$ is the total number of documents.

**TF-IDF Score:**
$$\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)$$

### Intuition:
- **High TF-IDF**: Term is frequent in this document but rare overall → **Important**
- **Low TF-IDF**: Term is either rare in this document or common everywhere → **Less important**

### Finance Application:
- Words like "stock", "market" appear everywhere → Low IDF
- Company-specific terms or events → High TF-IDF
- Helps identify what makes a document unique

In [ ]:
# TF-IDF Implementation

# Create TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=20,
    ngram_range=(1, 2)  # Include bigrams
)

tfidf_matrix = tfidf_vectorizer.fit_transform(financial_headlines)

# Display results
tfidf_features = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_features,
    index=[f"Doc_{i+1}" for i in range(len(financial_headlines))]
)

print("=" * 60)
print("TF-IDF REPRESENTATION")
print("=" * 60)
print(f"\n📊 Vocabulary Size: {len(tfidf_features)}")
print(f"📊 Matrix Shape: {tfidf_matrix.shape}")

print("\n🔤 Top Features (includes bigrams):")
print(list(tfidf_features))

# Show TF-IDF scores rounded for readability
print("\n📋 TF-IDF Matrix (rounded to 2 decimals):")
display(tfidf_df.round(2))

# Find most important terms per document
print("\n🏆 Most Important Terms per Document:")
for i, headline in enumerate(financial_headlines):
    row = tfidf_df.iloc[i]
    top_terms = row.nlargest(3)
    print(f"\nDoc {i+1}: {headline}")
    print(f"   Top terms: {', '.join([f'{term}({score:.2f})' for term, score in top_terms.items()])}")

## 2.3 Word Embeddings

Word embeddings represent words as dense vectors in a continuous space where semantically similar words are close together.

### Evolution of Word Representations:

| Method | Year | Dimensionality | Context |
|--------|------|----------------|---------|
| One-Hot | Classic | V (vocabulary size) | None |
| TF-IDF | Classic | V | Document-level |
| **Word2Vec** | 2013 | 50-300 | Local window |
| **GloVe** | 2014 | 50-300 | Global co-occurrence |
| **FastText** | 2016 | 50-300 | Subword |
| **BERT** | 2018 | 768+ | Full context (bidirectional) |

### Word2Vec Architectures:

**1. CBOW (Continuous Bag of Words)**
- Predicts target word from context words
- Faster to train
- Better for frequent words

**2. Skip-gram**
- Predicts context words from target word
- Better for rare words
- More commonly used

### Training Objective (Skip-gram with Negative Sampling):

$$\mathcal{L} = \sum_{(w,c) \in D} \left[ \log \sigma(v_c \cdot v_w) + \sum_{i=1}^{k} \log \sigma(-v_{n_i} \cdot v_w) \right]$$

where:
- $(w, c)$ are word-context pairs
- $v_w$, $v_c$ are word and context vectors
- $n_i$ are negative samples
- $\sigma$ is the sigmoid function

In [ ]:
# Word2Vec Demonstration (Conceptual)
# Note: Real Word2Vec requires large corpus. This demonstrates the concept.

# Simulated word embeddings for financial terms (3D for visualization)
np.random.seed(42)

# Create realistic financial word embedding clusters
financial_embeddings = {
    # Bullish terms (cluster 1)
    'bullish': np.array([0.8, 0.9, 0.7]),
    'surge': np.array([0.85, 0.95, 0.75]),
    'rally': np.array([0.75, 0.85, 0.8]),
    'growth': np.array([0.7, 0.8, 0.65]),
    'profit': np.array([0.9, 0.85, 0.7]),
    
    # Bearish terms (cluster 2)
    'bearish': np.array([-0.8, -0.7, 0.6]),
    'crash': np.array([-0.85, -0.75, 0.65]),
    'decline': np.array([-0.7, -0.65, 0.55]),
    'loss': np.array([-0.75, -0.8, 0.6]),
    'sell': np.array([-0.6, -0.55, 0.5]),
    
    # Neutral/Technical terms (cluster 3)
    'volatility': np.array([0.1, -0.1, -0.8]),
    'volume': np.array([0.15, 0.05, -0.75]),
    'spread': np.array([0.05, -0.05, -0.7]),
    'liquidity': np.array([0.0, 0.1, -0.85]),
}

# Calculate cosine similarity
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

print("=" * 60)
print("WORD EMBEDDING SIMILARITIES")
print("=" * 60)

# Find most similar words
def most_similar(word, embeddings, top_n=3):
    if word not in embeddings:
        return []
    
    word_vec = embeddings[word]
    similarities = []
    
    for other_word, other_vec in embeddings.items():
        if other_word != word:
            sim = cosine_similarity(word_vec, other_vec)
            similarities.append((other_word, sim))
    
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

# Test similarity
test_words = ['bullish', 'crash', 'volatility']
for word in test_words:
    similar = most_similar(word, financial_embeddings)
    print(f"\n📊 Most similar to '{word}':")
    for sim_word, score in similar:
        print(f"   {sim_word}: {score:.3f}")

# Word analogies demonstration
print("\n" + "=" * 60)
print("WORD ANALOGIES (king - man + woman = queen)")
print("=" * 60)

# bullish - profit + loss ≈ bearish?
result = financial_embeddings['bullish'] - financial_embeddings['profit'] + financial_embeddings['loss']
print("\n🧮 'bullish' - 'profit' + 'loss' = ?")

# Find closest word to result
closest = [(w, cosine_similarity(result, v)) for w, v in financial_embeddings.items()]
closest = sorted(closest, key=lambda x: x[1], reverse=True)[:3]
print(f"   Result closest to: {closest[0][0]} (similarity: {closest[0][1]:.3f})")

In [ ]:
# Visualize Word Embeddings in 2D
from sklearn.decomposition import PCA

# Prepare data for visualization
words = list(financial_embeddings.keys())
vectors = np.array([financial_embeddings[w] for w in words])

# Reduce to 2D using PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(vectors)

# Create visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Color by category
colors = {'bullish': 'green', 'bearish': 'red', 'neutral': 'blue'}
word_colors = []
for word in words:
    if word in ['bullish', 'surge', 'rally', 'growth', 'profit']:
        word_colors.append('green')
    elif word in ['bearish', 'crash', 'decline', 'loss', 'sell']:
        word_colors.append('red')
    else:
        word_colors.append('blue')

scatter = ax.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c=word_colors, s=200, alpha=0.6)

# Add labels
for i, word in enumerate(words):
    ax.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), 
                fontsize=12, ha='center', va='bottom',
                fontweight='bold')

ax.set_xlabel('PCA Component 1', fontsize=12)
ax.set_ylabel('PCA Component 2', fontsize=12)
ax.set_title('Financial Word Embeddings Visualization\n(Green=Bullish, Red=Bearish, Blue=Neutral)', 
             fontsize=14)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Semantically similar financial terms cluster together!")

---

# Part 3: Sentiment Analysis for Finance

## 3.1 Overview of Sentiment Analysis Approaches

Sentiment analysis extracts subjective information (opinions, emotions) from text.

### Approaches:

| Approach | Method | Pros | Cons |
|----------|--------|------|------|
| **Dictionary-Based** | Predefined word lists | Simple, interpretable | Domain-specific issues |
| **ML-Based** | Train classifier on labeled data | Learns from data | Needs training data |
| **Deep Learning** | Neural networks (BERT, etc.) | State-of-the-art accuracy | Computationally expensive |

### Financial Sentiment Dictionaries:

1. **Loughran-McDonald (LM)** - Specifically designed for financial text
   - 354 negative words (e.g., "loss", "decline", "adverse")
   - 75 positive words (e.g., "gain", "profit", "growth")
   
2. **Harvard General Inquirer (GI)** - General purpose
   - Not optimal for finance (e.g., "liability" is negative in general, neutral in finance)

3. **VADER** - Social media optimized
   - Handles emoticons, slang, intensifiers
   - Good for tweets, Reddit posts

> **💡 Key Insight**: Words have different meanings in finance!
> - "Liability" → Negative (general) vs Neutral (finance)
> - "Risk" → Negative (general) vs Neutral/Technical (finance)

In [ ]:
# Dictionary-Based Sentiment Analysis

# Simplified Loughran-McDonald Dictionary
lm_positive = {
    'achieve', 'advantage', 'beneficial', 'benefit', 'best', 'better', 
    'breakthrough', 'efficient', 'enhance', 'excellent', 'exceptional',
    'favorable', 'gain', 'good', 'great', 'growth', 'highest', 'improve',
    'improvement', 'innovation', 'opportunity', 'optimal', 'outperform',
    'positive', 'profit', 'profitable', 'progress', 'prosper', 'strength',
    'strong', 'succeed', 'success', 'successful', 'superior', 'surpass'
}

lm_negative = {
    'abandon', 'adverse', 'against', 'bankruptcy', 'catastrophe', 'concern',
    'concerns', 'crisis', 'critical', 'decline', 'decrease', 'default',
    'deficit', 'delay', 'deteriorate', 'difficult', 'difficulty', 'disappoint',
    'disappointing', 'downgrade', 'downturn', 'failure', 'fall', 'falling',
    'fear', 'impair', 'impairment', 'investigation', 'lawsuit', 'litigation',
    'loss', 'losses', 'negative', 'problem', 'problems', 'recession', 'risk',
    'risky', 'slowdown', 'terminate', 'threat', 'unable', 'uncertain',
    'uncertainty', 'unfavorable', 'weak', 'weakness', 'worse', 'worst'
}

def lm_sentiment_score(text):
    """
    Calculate sentiment score using Loughran-McDonald dictionary.
    Score = (Positive - Negative) / Total Words
    """
    words = text.lower().split()
    
    positive_count = sum(1 for w in words if w in lm_positive)
    negative_count = sum(1 for w in words if w in lm_negative)
    total_words = len(words)
    
    if total_words == 0:
        return 0, 0, 0, 0
    
    # Normalized sentiment score
    sentiment_score = (positive_count - negative_count) / total_words
    
    return sentiment_score, positive_count, negative_count, total_words

# Test on financial headlines
test_headlines = [
    "Company reports strong growth and excellent quarterly profit",
    "Stock crashes amid bankruptcy concerns and declining sales",
    "Fed maintains neutral stance on interest rates",
    "Tesla faces production problems but shows improvement in deliveries",
    "Apple achieves breakthrough success with new product innovation"
]

print("=" * 70)
print("LOUGHRAN-MCDONALD SENTIMENT ANALYSIS")
print("=" * 70)

results = []
for headline in test_headlines:
    score, pos, neg, total = lm_sentiment_score(headline)
    results.append({
        'Headline': headline[:50] + '...' if len(headline) > 50 else headline,
        'Positive': pos,
        'Negative': neg,
        'Score': score,
        'Sentiment': 'Positive' if score > 0.05 else ('Negative' if score < -0.05 else 'Neutral')
    })

results_df = pd.DataFrame(results)
display(results_df)

## 3.2 VADER Sentiment Analysis

**VADER** (Valence Aware Dictionary and sEntiment Reasoner) is a rule-based sentiment analyzer specifically designed for social media.

### Key Features:
- Handles **emoticons**: 😊, 😢
- Handles **intensifiers**: "extremely good" vs "good"
- Handles **negation**: "not good"
- Handles **punctuation**: "good!" vs "good"
- Handles **capitalization**: "GOOD" vs "good"

### VADER Output:
- `neg`: Negative proportion (0-1)
- `neu`: Neutral proportion (0-1)  
- `pos`: Positive proportion (0-1)
- `compound`: Normalized score (-1 to +1)

### Compound Score Interpretation:
- **compound >= 0.05**: Positive
- **compound <= -0.05**: Negative
- **-0.05 < compound < 0.05**: Neutral

In [ ]:
# VADER Sentiment Analysis
# Note: In production, use: from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Simplified VADER-like implementation for demonstration
class SimpleVADER:
    """
    Simplified VADER-like sentiment analyzer for demonstration.
    Real VADER has 7,500+ lexicon entries with valence scores.
    """
    
    def __init__(self):
        # Simplified lexicon with valence scores (-4 to +4)
        self.lexicon = {
            # Positive words
            'good': 1.9, 'great': 3.1, 'excellent': 3.2, 'amazing': 3.1,
            'profit': 2.0, 'growth': 1.8, 'surge': 2.5, 'rally': 2.3,
            'beat': 2.0, 'outperform': 2.5, 'bullish': 2.8, 'success': 2.5,
            'improve': 1.8, 'gain': 2.0, 'breakthrough': 2.8, 'innovation': 2.0,
            
            # Negative words
            'bad': -1.9, 'terrible': -3.0, 'awful': -3.0, 'poor': -1.8,
            'loss': -2.0, 'decline': -1.8, 'crash': -3.0, 'plunge': -2.8,
            'miss': -1.5, 'underperform': -2.0, 'bearish': -2.5, 'fail': -2.5,
            'concern': -1.5, 'risk': -1.0, 'crisis': -3.0, 'bankruptcy': -3.5,
            'lawsuit': -2.0, 'investigation': -2.0, 'warning': -1.8,
        }
        
        # Intensifiers
        self.intensifiers = {
            'very': 1.5, 'extremely': 2.0, 'incredibly': 1.8, 'really': 1.3,
            'slightly': 0.5, 'somewhat': 0.7
        }
        
        # Negation words
        self.negations = {'not', 'no', "n't", 'never', 'neither', 'nobody', 'nothing'}
    
    def polarity_scores(self, text):
        """Calculate sentiment scores for text."""
        words = text.lower().split()
        
        total_valence = 0
        word_count = 0
        
        for i, word in enumerate(words):
            # Clean word
            word = ''.join(c for c in word if c.isalpha())
            
            if word in self.lexicon:
                valence = self.lexicon[word]
                
                # Check for negation (within 3 words before)
                for j in range(max(0, i-3), i):
                    prev_word = ''.join(c for c in words[j] if c.isalpha())
                    if prev_word in self.negations:
                        valence *= -0.5
                        break
                
                # Check for intensifiers
                if i > 0:
                    prev_word = ''.join(c for c in words[i-1] if c.isalpha())
                    if prev_word in self.intensifiers:
                        valence *= self.intensifiers[prev_word]
                
                total_valence += valence
                word_count += 1
        
        # Normalize compound score to [-1, 1]
        compound = total_valence / (np.sqrt(total_valence**2 + 15)) if total_valence != 0 else 0
        
        # Calculate proportions
        if word_count == 0:
            return {'neg': 0, 'neu': 1, 'pos': 0, 'compound': 0}
        
        pos_prop = max(0, compound) 
        neg_prop = max(0, -compound)
        neu_prop = 1 - (pos_prop + neg_prop)
        
        return {
            'neg': round(neg_prop, 3),
            'neu': round(neu_prop, 3),
            'pos': round(pos_prop, 3),
            'compound': round(compound, 4)
        }

# Test VADER
vader = SimpleVADER()

social_media_texts = [
    "$AAPL is looking extremely bullish! 🚀",
    "Not good news for Tesla investors today",
    "Very concerned about the market crash",
    "BREAKING: Company reports amazing profit growth!!!",
    "This stock is not terrible but not great either",
    "Extremely bearish on crypto right now"
]

print("=" * 70)
print("VADER SENTIMENT ANALYSIS")
print("=" * 70)

vader_results = []
for text in social_media_texts:
    scores = vader.polarity_scores(text)
    sentiment = 'Positive' if scores['compound'] >= 0.05 else ('Negative' if scores['compound'] <= -0.05 else 'Neutral')
    vader_results.append({
        'Text': text[:45] + '...' if len(text) > 45 else text,
        'Compound': scores['compound'],
        'Pos': scores['pos'],
        'Neg': scores['neg'],
        'Neu': scores['neu'],
        'Sentiment': sentiment
    })

vader_df = pd.DataFrame(vader_results)
display(vader_df)

## 3.3 Machine Learning Sentiment Classification

For better accuracy, we can train ML classifiers on labeled financial text data.

### Common Approaches:

| Model | Features | Pros | Cons |
|-------|----------|------|------|
| **Naive Bayes** | TF-IDF | Fast, good baseline | Independence assumption |
| **Logistic Regression** | TF-IDF | Interpretable, regularizable | Linear boundaries |
| **SVM** | TF-IDF | Good with high-dimensional data | Slow on large datasets |
| **Random Forest** | TF-IDF | Handles non-linearity | Memory intensive |
| **BERT/FinBERT** | Contextual embeddings | State-of-the-art | Computationally expensive |

### Training Process:
1. Collect labeled financial text (positive/negative/neutral)
2. Preprocess text (tokenize, clean, normalize)
3. Create features (TF-IDF, embeddings)
4. Train classifier
5. Evaluate on held-out test set

In [ ]:
# Building a Sentiment Classifier

# Create synthetic labeled dataset
np.random.seed(42)

# Labeled financial headlines dataset
labeled_data = [
    # Positive (1)
    ("Apple beats earnings expectations with record iPhone sales", 1),
    ("Tesla stock surges 10% on strong delivery numbers", 1),
    ("Company reports exceptional profit growth this quarter", 1),
    ("Investors celebrate breakthrough innovation announcement", 1),
    ("Market rallies on positive economic data", 1),
    ("Strong quarterly results exceed analyst forecasts", 1),
    ("Revenue growth accelerates beyond expectations", 1),
    ("Share price hits all-time high on bullish outlook", 1),
    ("Company announces successful product launch", 1),
    ("Profit margins improve significantly year over year", 1),
    
    # Negative (0)
    ("Stock crashes amid bankruptcy concerns", 0),
    ("Company faces lawsuit over accounting fraud", 0),
    ("Revenue decline raises investor concerns", 0),
    ("Market plunges on recession fears", 0),
    ("Investigation launched into financial irregularities", 0),
    ("Quarterly loss exceeds analyst expectations", 0),
    ("Downgrade sparks massive sell-off", 0),
    ("Production problems lead to delivery delays", 0),
    ("Company warns of declining demand", 0),
    ("Share price collapses on poor earnings report", 0),
    
    # Neutral (2)
    ("Fed maintains current interest rate policy", 2),
    ("Company announces quarterly earnings call date", 2),
    ("Stock trades sideways amid low volume", 2),
    ("Analyst maintains hold rating on shares", 2),
    ("Company completes previously announced acquisition", 2),
    ("Board appoints new chief financial officer", 2),
    ("Dividend payment date set for next month", 2),
    ("Company files annual report with SEC", 2),
]

# Prepare data
texts = [t[0] for t in labeled_data]
labels = [t[1] for t in labeled_data]

# Create TF-IDF features
tfidf = TfidfVectorizer(stop_words='english', max_features=100, ngram_range=(1, 2))
X = tfidf.fit_transform(texts)
y = np.array(labels)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("=" * 60)
print("ML SENTIMENT CLASSIFIER")
print("=" * 60)
print(f"\n📊 Dataset size: {len(texts)}")
print(f"📊 Training size: {X_train.shape[0]}")
print(f"📊 Test size: {X_test.shape[0]}")
print(f"📊 Feature dimensions: {X.shape[1]}")
print(f"📊 Class distribution: {dict(Counter(labels))}")

# Train classifiers
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    results[name] = {'train_acc': train_acc, 'test_acc': test_acc}
    print(f"\n🤖 {name}:")
    print(f"   Training Accuracy: {train_acc:.2%}")
    print(f"   Test Accuracy: {test_acc:.2%}")

In [ ]:
# Feature Importance Analysis (Logistic Regression)

# Get feature names and coefficients
feature_names = tfidf.get_feature_names_out()
lr_model = models['Logistic Regression']

# For multiclass, we look at coefficients per class
# Class 0 = Negative, Class 1 = Positive, Class 2 = Neutral
class_names = ['Negative', 'Positive', 'Neutral']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, (class_name, ax) in enumerate(zip(class_names, axes)):
    coef = lr_model.coef_[idx]
    
    # Get top features
    top_positive_idx = np.argsort(coef)[-10:]
    top_negative_idx = np.argsort(coef)[:10]
    
    top_idx = np.concatenate([top_negative_idx, top_positive_idx])
    top_features = feature_names[top_idx]
    top_coefs = coef[top_idx]
    
    colors = ['red' if c < 0 else 'green' for c in top_coefs]
    
    ax.barh(range(len(top_idx)), top_coefs, color=colors, alpha=0.7)
    ax.set_yticks(range(len(top_idx)))
    ax.set_yticklabels(top_features)
    ax.set_xlabel('Coefficient')
    ax.set_title(f'{class_name} Class Features')
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.3)

plt.suptitle('Most Important Features by Sentiment Class', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Test on new headlines
print("\n" + "=" * 60)
print("PREDICTION ON NEW HEADLINES")
print("=" * 60)

new_headlines = [
    "Bitcoin surges to new all-time high amid institutional buying",
    "Company faces regulatory investigation into business practices",
    "Quarterly earnings report scheduled for next Tuesday"
]

new_features = tfidf.transform(new_headlines)
predictions = lr_model.predict(new_features)
probabilities = lr_model.predict_proba(new_features)

for headline, pred, probs in zip(new_headlines, predictions, probabilities):
    sentiment = class_names[pred]
    print(f"\n📰 \"{headline}\"")
    print(f"   Prediction: {sentiment}")
    print(f"   Probabilities: Neg={probs[0]:.2%}, Pos={probs[1]:.2%}, Neu={probs[2]:.2%}")

## 3.4 Modern Approaches: FinBERT

**FinBERT** is a BERT model fine-tuned on financial text for sentiment analysis.

### Architecture Overview:

```
Input Text → Tokenization → BERT Encoder → [CLS] Token → Classification Head → Sentiment
```

### Why FinBERT?
1. **Pre-trained on financial corpus**: Understands financial jargon
2. **Contextual embeddings**: Same word can have different meanings
3. **Transfer learning**: Leverages knowledge from massive text corpora

### FinBERT vs Traditional Methods:

| Aspect | Dictionary | ML + TF-IDF | FinBERT |
|--------|------------|-------------|---------|
| Context | ❌ None | ❌ Limited | ✅ Full |
| Domain-specific | ⚠️ Manual | ⚠️ Training data | ✅ Pre-trained |
| Negation | ⚠️ Rules | ⚠️ Learned | ✅ Contextual |
| Accuracy | Low-Medium | Medium | High |
| Speed | Fast | Fast | Slow |
| Resources | Low | Low | High (GPU) |

### Example Usage (Conceptual):
```python
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load FinBERT
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

# Predict
inputs = tokenizer("Apple stock surges on earnings beat", return_tensors="pt")
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# Output: [negative_prob, neutral_prob, positive_prob]
```

---

# Part 4: Alternative Data

## 4.1 What is Alternative Data?

**Alternative data** refers to non-traditional data sources used to generate alpha or improve risk management in financial markets.

### Traditional vs Alternative Data:

| Traditional Data | Alternative Data |
|------------------|------------------|
| Price, volume | Satellite imagery |
| Financial statements | Credit card transactions |
| Economic indicators | Web traffic |
| Analyst reports | Social media sentiment |
| Earnings calls | Geolocation data |
| SEC filings | App usage |

### The Alternative Data Landscape:

```
┌─────────────────────────────────────────────────────────────────┐
│                    ALTERNATIVE DATA UNIVERSE                     │
├─────────────────┬─────────────────┬─────────────────────────────┤
│    EXHAUST      │    SENSORS      │      WEB/DIGITAL           │
│    DATA         │    DATA         │      DATA                  │
├─────────────────┼─────────────────┼─────────────────────────────┤
│ • Credit cards  │ • Satellite     │ • News sentiment           │
│ • POS data      │ • Weather       │ • Social media             │
│ • Email receipts│ • IoT devices   │ • Web scraping             │
│ • App usage     │ • Foot traffic  │ • Search trends            │
│ • Job postings  │ • Ship tracking │ • Product reviews          │
└─────────────────┴─────────────────┴─────────────────────────────┘
```

In [ ]:
# Alternative Data Sources Overview

alt_data_sources = {
    'Category': [
        'Satellite Imagery', 'Satellite Imagery', 
        'Web/Social', 'Web/Social', 'Web/Social',
        'Transaction', 'Transaction',
        'Geolocation', 'Geolocation',
        'Other', 'Other'
    ],
    'Data Type': [
        'Parking lot cars', 'Oil storage tanks',
        'News sentiment', 'Twitter mentions', 'Google Trends',
        'Credit card spend', 'Email receipts',
        'Foot traffic', 'Ship tracking (AIS)',
        'Job postings', 'Patent filings'
    ],
    'Use Case': [
        'Retail sales prediction', 'Oil inventory levels',
        'Market sentiment', 'Brand sentiment', 'Interest trends',
        'Consumer spending', 'E-commerce tracking',
        'Store visits', 'Commodity supply',
        'Company growth', 'Innovation tracking'
    ],
    'Alpha Decay': [
        'Days', 'Days',
        'Hours-Days', 'Minutes-Hours', 'Days',
        'Weeks', 'Weeks',
        'Days-Weeks', 'Days',
        'Weeks-Months', 'Months'
    ],
    'Example Provider': [
        'Orbital Insight', 'Kayrros',
        'RavenPack', 'StockTwits', 'Google',
        'Second Measure', 'Earnest Research',
        'SafeGraph', 'MarineTraffic',
        'LinkUp', 'PatSnap'
    ]
}

alt_df = pd.DataFrame(alt_data_sources)

print("=" * 80)
print("ALTERNATIVE DATA LANDSCAPE")
print("=" * 80)
display(alt_df)

# Visualize by category
fig, ax = plt.subplots(figsize=(10, 6))
category_counts = alt_df['Category'].value_counts()
colors = plt.cm.Set3(range(len(category_counts)))
wedges, texts, autotexts = ax.pie(category_counts, labels=category_counts.index, 
                                   autopct='%1.0f%%', colors=colors)
ax.set_title('Alternative Data by Category', fontsize=14)
plt.show()